# 📊 Preparing Tableau Dashboards for Credit Risk Insights

In this final notebook, I prepare targeted CSV files from our validation dataset and SHAP outputs to support **interactive Tableau dashboards**. These dashboards are designed to communicate key insights from the credit risk modeling pipeline to both technical and non-technical audiences.

### 🎯 Objectives

- Export curated datasets to power five core Tableau visualizations:
  1. SHAP summary statistics for global feature importance
  2. Risk distribution by predicted probability
  3. Feature importance by applicant group (e.g., default vs non-default)
  4. Confusion matrix with precision/recall metrics
  5. Individual applicant-level SHAP values for drill-down explanations

Each export is tailored to maximize clarity, interactivity, and storytelling impact inside Tableau Public.

> This notebook acts as the **bridge between machine learning outputs and stakeholder communication**, enabling the delivery of interpretable, transparent credit risk insights.

### 📦 Load Required Libraries

I begin by importing the essential libraries for this final notebook.

In [8]:
# Core data manipulation
import pandas as pd
import numpy as np

# Model loading and evaluation tools
from sklearn.model_selection import train_test_split
import joblib

# SHAP explanations and LightGBM model compatibility
import shap
import lightgbm as lgb

# Suppress SHAP warning about binary classifiers
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

### 🧠 Load Trained Model and Validation Data

I begin by loading the final trained LightGBM model along with the validation dataset and corresponding labels. These will be used to generate prediction outputs and SHAP values for Tableau-ready visualizations.

In [9]:
# Load the final trained LightGBM model
model = joblib.load("../models/lgbm_model.joblib")

# Load the validation feature set
X_valid = pd.read_parquet("../data/processed/X_valid.parquet")

# Load the corresponding validation labels
y_valid = pd.read_parquet("../data/processed/y_valid.parquet").squeeze()

### 📊 Prepare Global SHAP Summary for Tableau Dashboarding

In this section, I generate a summary of global feature importance using SHAP values and prepare it for visualization in Tableau.

- First, I predict loan default probabilities on the validation set and assign class labels based on the chosen threshold (0.3).
- I then compute **SHAP values** using `TreeExplainer`, which is optimized for our LightGBM model.
- To summarize global importance, we calculate the **mean absolute SHAP value per feature**, giving us a ranked list of the most influential variables.
- Finally, I save this summary to a CSV file (`global_shap_importance.csv`) in the `data/final/` directory. This file can be easily loaded into Tableau to build an interactive feature importance dashboard.

In [10]:
# Predict probabilities and assign predicted labels using chosen threshold
y_pred_proba = model.predict_proba(X_valid)[:, 1]
y_pred_thresh = (y_pred_proba >= 0.3).astype(int)

# Add prediction columns to X_valid
X_valid_final = X_valid.copy()
X_valid_final["loan_default_proba"] = y_pred_proba
X_valid_final["predicted_label"] = y_pred_thresh
X_valid_final["actual_label"] = y_valid.values

# Initialize SHAP explainer (TreeExplainer for LightGBM)
explainer = shap.TreeExplainer(model)

# Compute SHAP values (shape: [n_samples, n_features])
shap_values = explainer.shap_values(X_valid)

# Compute global SHAP feature importance (mean absolute value)
global_importance = (
    pd.DataFrame(shap_values, columns=X_valid.columns)
    .abs()
    .mean()
    .reset_index()
    .rename(columns={"index": "feature_name", 0: "mean_abs_shap_value"})
    .sort_values("mean_abs_shap_value", ascending=False)
)

# Save global SHAP importance summary for Tableau
global_importance.to_csv("../data/final/global_shap_importance.csv", index=False)
print("✅ Global SHAP feature importance saved to global_shap_importance.csv")

✅ Global SHAP feature importance saved to global_shap_importance.csv


### 📊 Save Risk Distribution Data for Tableau

To support visual analysis of loan default risk scores in Tableau, I prepare a dataset containing key applicant features and the model’s predicted probabilities:

- `loan_default_proba`: Model’s predicted probability of default
- `actual_label`: Ground truth indicating default or not
- `DAYS_BIRTH`, `AMT_INCOME_TOTAL`, `DAYS_EMPLOYED`: Key demographic and financial indicators
- `label`: Readable label version of the actual class (e.g., “Default”, “No Default”)

This file can be used to create histograms, density plots, or stratified risk profiles in Tableau dashboards.

In [11]:
# Add readable labels to indicate default vs no default
X_valid_final["label"] = X_valid_final["actual_label"].map({0: "No Default", 1: "Default"})

# Select relevant columns for visualization
risk_df = X_valid_final[[
    "loan_default_proba",
    "actual_label",
    "DAYS_BIRTH",
    "AMT_INCOME_TOTAL",
    "DAYS_EMPLOYED",
    "label"
]]

# Save to CSV
risk_df.to_csv("../data/final/risk_distribution.csv", index=False)
print("✅ Risk distribution saved to risk_distribution.csv")

✅ Risk distribution saved to risk_distribution.csv


In [2]:
shap_df = pd.read_csv("../data/final/shap_values_tableau.csv")

# 🏷️ Create risk bands from predicted probabilities
bins = [0, 0.2, 0.5, 1.0]
labels = ["Low Risk", "Medium Risk", "High Risk"]
shap_df["risk_band"] = pd.cut(shap_df["loan_default_proba"], bins=bins, labels=labels)

# 🔝 Top 15 features based on mean absolute SHAP value
mean_abs_shap = shap_df.drop(columns=["loan_default_proba", "predicted_label", "risk_band"]).abs().mean()
top_features = mean_abs_shap.sort_values(ascending=False).head(15).index.tolist()

# 📊 Melt and group for Tableau
shap_melted = shap_df[["risk_band"] + top_features].melt(id_vars="risk_band", var_name="feature", value_name="shap_value")
agg_df = shap_melted.groupby(["risk_band", "feature"], observed=True).mean().reset_index()

# 💾 Save for Tableau
agg_df.to_csv("../data/final/agg_shap_by_risk_band.csv", index=False)
print("✅ SHAP aggregation saved for Tableau!")

✅ SHAP aggregation saved for Tableau!


In [10]:
# Purpose: Prepare SHAP vs Risk Score visualization data in long format for Tableau

import pandas as pd

# 1. Load the saved shap_vs_risk.csv
df = pd.read_csv("../data/final/shap_vs_risk.csv")

# 2. Define correct column names
feature_cols = [
    "EXT_SOURCE_1",
    "EXT_SOURCE_2",
    "EXT_SOURCE_3",
    "credit_annuity_ratio",
    "credit_goods_ratio",
    "CODE_GENDER_M",
    "DAYS_BIRTH",
    "ORGANIZATION_TYPE_TE"
]

# 3. Melt into long format
long_df = df.melt(
    id_vars=["loan_default_proba"],
    value_vars=feature_cols,
    var_name="Feature",
    value_name="SHAP Value"
)

# 4. Save to CSV for Tableau
long_df.to_csv("../data/final/shap_vs_risk_long.csv", index=False)
print("✅ Saved: shap_vs_risk_long.csv")

✅ Saved: shap_vs_risk_long.csv


In [7]:
# 🔄 Melt SHAP values from wide to long format
shap_long = shap_df.melt(
    id_vars=["loan_default_proba", "predicted_label", "actual_label"],
    var_name="feature",
    value_name="shap_value"
)

# 💾 Save for Tableau
shap_long.to_csv("../data/final/shap_values_long.csv", index=False)
print("✅ SHAP long-format file saved!")

✅ SHAP long-format file saved!


In [3]:
# 📦 Imports (in case not already)
import pandas as pd
from sklearn.metrics import confusion_matrix

# ✅ Calculate confusion matrix components
cm = confusion_matrix(X_valid_final["actual_label"], X_valid_final["predicted_label"])
tn, fp, fn, tp = cm.ravel()

# 🧾 Create a summary DataFrame
confusion_summary = pd.DataFrame({
    "Metric": ["True Negative", "False Positive", "False Negative", "True Positive"],
    "Count": [tn, fp, fn, tp]
})

# 💾 Save the confusion matrix summary
confusion_summary.to_csv("../data/final/confusion_matrix_summary.csv", index=False)
print("✅ Saved: confusion_matrix_summary.csv")

✅ Saved: confusion_matrix_summary.csv


In [4]:
# 🧮 Save row-level predictions
X_valid_final[["loan_default_proba", "predicted_label", "actual_label"]].to_csv(
    "../data/final/confusion_predictions.csv", index=False
)
print("✅ Saved: confusion_predictions.csv")

✅ Saved: confusion_predictions.csv


In [8]:
import pandas as pd
import numpy as np

# Load your two CSVs
summary_df = pd.read_csv("../data/final/confusion_matrix_summary.csv")
pred_df = pd.read_csv("../data/final/confusion_predictions.csv")

# Add missing columns with explicit types
summary_df["loan_default_proba"] = pd.Series([np.nan] * len(summary_df), dtype="float64")
summary_df["actual_label"] = pd.Series([np.nan] * len(summary_df), dtype="float64")
summary_df["predicted_label"] = pd.Series([np.nan] * len(summary_df), dtype="float64")
summary_df["Source"] = "Summary"

pred_df["Metric"] = pd.Series([pd.NA] * len(pred_df), dtype="string")
pred_df["Count"] = pd.Series([np.nan] * len(pred_df), dtype="float64")
pred_df["Source"] = "Prediction"

# Define consistent column order
combined_cols = ["Metric", "Count", "loan_default_proba", "actual_label", "predicted_label", "Source"]

# Reorder and ensure alignment
summary_part = summary_df[combined_cols]
pred_part = pred_df[combined_cols]

# Concatenate without warning
combined_df = pd.concat([summary_part, pred_part], ignore_index=True)

# Save
combined_df.to_csv("../data/final/confusion_combined.csv", index=False)
print("✅ Combined file saved with consistent dtypes and no warnings.")

✅ Combined file saved with consistent dtypes and no warnings.


In [9]:
import pandas as pd

df = pd.read_csv("../data/final/confusion_combined.csv")

print("🔢 Shape:", df.shape)
print("🧾 Column names:", df.columns.tolist())
print("📌 Sample rows:")
print(df.head(10))
print("🎯 Unique values in Actual and Predicted Labels:")
print("  - Actual:", df["actual_label"].unique())
print("  - Predicted:", df["predicted_label"].unique())

🔢 Shape: (61507, 6)
🧾 Column names: ['Metric', 'Count', 'loan_default_proba', 'actual_label', 'predicted_label', 'Source']
📌 Sample rows:
           Metric    Count  loan_default_proba  actual_label  predicted_label  \
0   True Negative  55095.0                 NaN           NaN              NaN   
1  False Positive   1443.0                 NaN           NaN              NaN   
2  False Negative   4122.0                 NaN           NaN              NaN   
3   True Positive    843.0                 NaN           NaN              NaN   
4             NaN      NaN            0.032648           0.0              0.0   
5             NaN      NaN            0.060563           0.0              0.0   
6             NaN      NaN            0.282761           0.0              0.0   
7             NaN      NaN            0.076722           0.0              0.0   
8             NaN      NaN            0.081543           0.0              0.0   
9             NaN      NaN            0.078865      

In [10]:
df = pd.read_csv("../data/final/confusion_combined.csv")

# ✅ Keep only the prediction rows
df_pred = df[df["Source"] == "Prediction"].copy()

# ✅ Save clean version for Tableau
df_pred.to_csv("../data/final/confusion_prediction_only.csv", index=False)
print("✅ Cleaned file saved!")

✅ Cleaned file saved!


In [12]:
import pandas as pd

df = pd.read_csv("../data/final/confusion_prediction_only.csv")

print("🔢 Shape:", df.shape)
print("🧾 Column names:", df.columns.tolist())
print("📌 Sample rows:")
print(df.head(10))
print("🎯 Unique values in Actual and Predicted Labels:")
print("  - Actual:", df["actual_label"].unique())
print("  - Predicted:", df["predicted_label"].unique())

🔢 Shape: (61503, 6)
🧾 Column names: ['Metric', 'Count', 'loan_default_proba', 'actual_label', 'predicted_label', 'Source']
📌 Sample rows:
   Metric  Count  loan_default_proba  actual_label  predicted_label  \
0     NaN    NaN            0.032648           0.0              0.0   
1     NaN    NaN            0.060563           0.0              0.0   
2     NaN    NaN            0.282761           0.0              0.0   
3     NaN    NaN            0.076722           0.0              0.0   
4     NaN    NaN            0.081543           0.0              0.0   
5     NaN    NaN            0.078865           0.0              0.0   
6     NaN    NaN            0.004224           0.0              0.0   
7     NaN    NaN            0.004712           0.0              0.0   
8     NaN    NaN            0.402899           0.0              1.0   
9     NaN    NaN            0.087414           1.0              0.0   

       Source  
0  Prediction  
1  Prediction  
2  Prediction  
3  Prediction  


In [14]:
import pandas as pd

# Load predictions (assuming you already have these)
# y_valid = ground truth labels
# y_pred_thresh = predicted labels (0 or 1)

# Define confusion matrix components
TP = ((y_valid == 1) & (y_pred_thresh == 1)).sum()
TN = ((y_valid == 0) & (y_pred_thresh == 0)).sum()
FP = ((y_valid == 0) & (y_pred_thresh == 1)).sum()
FN = ((y_valid == 1) & (y_pred_thresh == 0)).sum()

# Create summary DataFrame
summary_df = pd.DataFrame({
    "Metric": ["True Positive", "False Positive", "False Negative", "True Negative"],
    "Count": [TP, FP, FN, TN],
    "loan_default_proba": [pd.NA] * 4,
    "actual_label": [pd.NA] * 4,
    "predicted_label": [pd.NA] * 4,
    "Source": ["Summary"] * 4
})

# Save to CSV to be used in Tableau
summary_df.to_csv("../data/final/confusion_summary.csv", index=False)
print("✅ Summary saved: confusion_summary.csv")

✅ Summary saved: confusion_summary.csv
